In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# **Data Import & Cleaning Hypotheses**

## **1. Import-Time Auto-Corrections**  
*(Automatically handled during data loading)*  

| **Issue Type**               | **Correction Applied**          | **Preserved Rows** | **Impact Analysis**                   |
|-------------------------------|----------------------------------|--------------------|---------------------------------------|
| Negative Inventory            | Clip to 0                       | Yes                | May mask true stockouts               |
| Sales > Inventory             | Cap at Inventory Level          | Yes                | Preserves volume but loses overflow   |
| Invalid Dates                 | Drop rows                       | No                 | Creates time-series gaps              |
| Numeric Overflow              | Downcast to unsigned            | Yes                | Reduces memory usage                  |

**Hypothesis**:  
> "Automated corrections during import will preserve 98%+ of rows while creating traceable artifacts for analysis."

---

## **2. Post-Import Quality Checks**  
*(Require manual validation)*  

| **Check Type**          | **Detection Method**             | **Potential Actions**                  |
|-------------------------|-----------------------------------|----------------------------------------|
| Label Consistency       | Fuzzy matching on text fields    | Standardize categories/regions         |
| Temporal Gaps           | Missing date detection           | Interpolate or flag as special events  |
| Price Inflation         | 3σ deviation from product history| Verify true inflation vs data errors   |
| Promotion Efficacy      | Lift analysis (promo vs non-promo)| Filter phantom demand from stockouts   |

**Hypothesis**:  
> "5-15% of rows will require post-import adjustments, mainly in categorical labels and promotional periods."

---

## **3. Feature Engineering Implications**

| **Observed Pattern**       | **Feature Design**               | **Rationale**                         |
|----------------------------|-----------------------------------|---------------------------------------|
| Frequent zero-demand       | Demand presence indicator        | Supports intermittent-demand models   |
| Epidemic spikes            | Shock absorption features        | Isolates exceptional events           |
| Regional price variance    | Region-price clusters            | Captures local market dynamics        |

**Hypothesis**:  
> "Auto-corrected inventory/sales relationships will require demand reconstruction features."

In [ ]:
# ==========================================================================
# Step A.1: Data Import and Proactive Cleaning
# Purpose: To import the retail sales data in chunks while applying automated corrections for common data
#          integrity issues and tracking all modifications, as outlined in the initial hypotheses.
# Outputs: A cleaned DataFrame, a log of any import errors, and detailed statistics on dropped or modified rows.
# Rationale: This foundational step ensures that the raw data is loaded efficiently and that baseline
#            data quality is enforced from the start, providing a clean and well-documented dataset for analysis.
# ==========================================================================

# Imports and Configuration
from pathlib import Path
import pandas as pd
import numpy as np

DATA_PATH = Path('/kaggle/input/retail-store-inventory-and-demand-forecasting/sales_data.csv')
CHUNKSIZE = 50_000
DATE_COL = 'Date'

# Optimized dtypes
DTYPES = {
    'Store ID': 'category',
    'Product ID': 'category',
    'Category': 'category',
    'Region': 'category',
    'Inventory Level': 'uint16',
    'Units Sold': 'uint16',
    'Units Ordered': 'uint16',
    'Price': 'float32',
    'Discount': 'float32',
    'Weather Condition': 'category',
    'Promotion': 'bool',
    'Competitor Pricing': 'float32',
    'Seasonality': 'category',
    'Epidemic': 'bool',
    'Demand': 'uint16'
}

def import_retail_data():
    """
    Imports retail sales data in chunks, applies corrections, and tracks changes.
    Returns cleaned data, error messages, and processing statistics.
    """
    chunks = []
    error_log = []
    stats = {
        'total_rows': 0,
        'rows_dropped': {
            'date_parsing': 0,
            'dtype_conversion': 0,
            'chunk_failures': 0
        },
        'rows_modified': {
            'negative_inventory': 0,
            'sales_exceeding_inventory': 0
        }
    }

    try:
        with pd.read_csv(
            DATA_PATH,
            chunksize=CHUNKSIZE,
            dtype=DTYPES,
            parse_dates=[DATE_COL],
            on_bad_lines='warn',
            encoding='utf-8'
        ) as reader:
            for chunk_idx, chunk in enumerate(reader):
                stats['total_rows'] += len(chunk)
                try:
                    # Validate required columns
                    missing_cols = set(DTYPES.keys()) - set(chunk.columns)
                    if missing_cols:
                        raise ValueError(f"Missing columns: {missing_cols}")

                    # Track modifications
                    stats['rows_modified']['negative_inventory'] += (chunk['Inventory Level'] < 0).sum()
                    stats['rows_modified']['sales_exceeding_inventory'] += (chunk['Units Sold'] > chunk['Inventory Level']).sum()

                    # Clean data - Date parsing
                    chunk[DATE_COL] = pd.to_datetime(chunk[DATE_COL], errors='coerce')
                    date_mask = chunk[DATE_COL].isna()
                    stats['rows_dropped']['date_parsing'] += date_mask.sum()
                    chunk = chunk[~date_mask]

                    # Clean data - Inventory adjustment
                    chunk['Inventory Level'] = chunk['Inventory Level'].clip(lower=0)

                    # Clean data - Sales adjustment
                    chunk['Units Sold'] = chunk[['Units Sold', 'Inventory Level']].min(axis=1)

                    chunks.append(chunk)

                except Exception as e:
                    stats['rows_dropped']['chunk_failures'] += len(chunk)
                    error_log.append(f"Chunk {chunk_idx} failed: {str(e)}")
                    continue

        if not chunks:
            raise ValueError("No valid data chunks were processed")

        df = pd.concat(chunks, ignore_index=False).set_index(DATE_COL).sort_index()
        stats['final_rows'] = len(df)

        return df, error_log, stats

    except Exception as e:
        error_log.append(f"Fatal import error: {str(e)}")
        return None, error_log, stats

# Execute import and log errors
df, import_errors, import_stats = import_retail_data()

In [ ]:
# ==========================================================================
# Step A.2: Report on Initial Data Import and Cleaning
# Purpose: To display the results from the proactive data import, showing the final row count, a summary of all
#          automated corrections and dropped rows, and a sample of the resulting DataFrame.
# Outputs: Printed statements summarizing the import process and a display of the head of the cleaned DataFrame.
# Rationale: This step provides immediate feedback on the success and impact of the initial data import and
#            cleaning phase, offering a transparent view of what changes were made automatically.
# ==========================================================================



if df is not None:
    print(f"Successfully imported {import_stats['final_rows']:,} rows (out of {import_stats['total_rows']:,} total)")
    
    print("\nRows dropped during processing:")
    for reason, count in import_stats['rows_dropped'].items():
        print(f"- {reason.replace('_', ' ').title()}: {count:,} rows")
    
    print("\nRows auto-corrected (not dropped):")
    for issue, count in import_stats['rows_modified'].items():
        print(f"- {issue.replace('_', ' ').title()}: {count:,} rows")
    
    print("\nSample data:")
    display(df.head())
else:
    print("Import failed")

if import_errors:
    print("\nEncountered warnings/errors:")
    for i, error in enumerate(import_errors, 1):
        print(f"{i}. {error}")
else:
    print("\nNo import errors detected.")

# Retail Data Quality Assessment Framework

---

## 1. Investigation Purpose

Systematically evaluate dataset quality to:

- **Identify integrity issues affecting forecast accuracy**
- **Uncover demand, sales, inventory, and regional patterns**
- **Guide model selection based on data characteristics**
- **Document critical metadata for reproducibility**

---

## 2. Key Investigation Areas

### A. Data Quality Checks

**Rationale:**  
Ensure reliable inputs for forecasting.

**Targets:**

1. **Null/NaN and missing data:**  
   Quantify and address gaps in key columns.
2. **Row completeness:**  
   Ensure all required columns are populated.
3. **Data type verification:**  
   Confirm correct types for each column.
4. **Unique identifiers:**  
   Check for duplicates in key fields.
5. **Category/label consistency:**  
   Detect typos, inconsistencies, and rare/catch-all labels.
6. **Date validity:**  
   Identify gaps or anomalous timestamps.
7. **Value validation:**  
   Flag artificial caps, placeholder values, or illogical relationships (e.g., sales > inventory, negative prices).
8. **Flag consistency:**  
   Validate binary flags (promotion, epidemic).

**Outputs:**

- **Counts of missing/incomplete/invalid values**
- **Visualizations:** Bar plots, heatmaps, histograms

---


### B. Dataset Dimensions

**Rationale:** Understand data granularity and coverage  
**Metrics:**

| Dimension         | Purpose                                                         |
|-------------------|-----------------------------------------------------------------|
| Product Count     | Determine SKU-level vs. category forecasting                    |
| Category Count    | Identify overbroad or inconsistent groupings                    |
| Date Range        | Assess seasonality coverage and temporal completeness           |
| Store Count       | Evaluate store-level completeness and granularity               |
| Region Count      | Assess geographical representation and region-level consistency |

---

### C. Demand Characteristics

**Rationale:** Inform model selection  
**Analysis:**

1. **Value range (min/max):** Detect censoring or artificial limits
2. **Zero-demand frequency:** Identify intermittent demand patterns
3. **Epidemic/promotion variance:** Assess shock resilience and event-driven demand
4. **Regional patterns:** Detect variance in sales/demand across regions

**Outputs:**

- **Summary statistics**
- **Distribution plots, time series with event markers, regional heatmaps**

---

## 3. Insights-to-Action Mapping

| Finding                | Impact                              | Action                                      |
|------------------------|-------------------------------------|---------------------------------------------|
| High zero-demand %     | Intermittent demand pattern         | Use Croston’s/TSB models                    |
| Regional sales clusters| Geographic bias                     | Add region dummy variables                  |
| Epidemic demand spikes | Non-stationary time series          | Implement shock indicator features          |
| Category/label typos   | Misattribution, inconsistent analysis| Consolidate labels pre-analysis             |
| Missing weather/pricing| Incomplete contextual factors       | Impute regionally or exclude weather-dependent models |
| Value anomalies        | Implausible results                 | Correct or flag for review                  |
| Flag inconsistencies   | Misleading uplift analysis          | Correct/validate flags                      |
| Negative inventory     | Implausible sales patterns          | Correct or flag for review                  |

---

## 4. Target Outputs

- **Data Quality Scorecard**
  - Summary metrics for each quality dimension
  - Visual dashboards of key findings
- **Priority Cleaning Tasks**
  - Roadmap for addressing identified issues
- **Model Compatibility Assessment**
  - Evaluation of data suitability for different forecasting models
- **Feature Engineering Roadmap**
  - Plan for creating derived features (e.g., region dummies, event flags)
- **Model Selection Justification**
  - Documentation of model choices based on data characteristics

---

In [ ]:
# ==========================================================================
# Step A: Foundational Data Quality Checks
# Purpose: To perform a comprehensive first-pass analysis of the dataset, identifying common quality issues
#          like missing values, incorrect data types, duplicates, invalid values, and inconsistent flags.
# Outputs: A consolidated report, visualizations for each check, and a dictionary of problematic row indices.
# Rationale: Establishing a baseline of data quality is critical before deeper analysis or modeling,
#            ensuring subsequent steps are built on a reliable foundation.
# ==========================================================================

import pandas as pd
import matplotlib.pyplot as plt

# ================================================================
# Reset index for checks: Make 'Date' a column for validation
# ================================================================
df_check = df.reset_index()

# Initialize dictionary to store problematic row indices
problem_rows = {
    'missing_any': set(),
    'missing_required': set(),
    'duplicate_composite': set(),
    'invalid_date': set(),
    'future_date': set(),
    'ancient_date': set(),
    'negative_inventory': set(),
    'negative_price': set(),
    'sales_exceeds_inventory': set(),
    'non_binary_promotion': set(),
    'non_binary_epidemic': set(),
    'rare_category': set(),
    'rare_region': set(),
    'rare_weather': set(),
    'rare_seasonality': set(),
}

# Initialize a string to collect all report text
report_text = ""

def log_report(message):
    """Helper function to accumulate report text."""
    global report_text
    report_text += message + "\n"

# -----------------------------------------------------------------
# 1. Null/NaN and Missing Data
# -----------------------------------------------------------------
try:
    missing_any = df_check[df_check.isna().any(axis=1)].index
    problem_rows['missing_any'].update(missing_any)
    log_report(f"Rows with any missing data: {len(missing_any)}")
    
    # --- Enhanced Plotting ---
    plt.figure(figsize=(12, 6))
    missing_counts = df_check.isna().sum()
    missing_counts = missing_counts[missing_counts > 0] # Only plot columns with missing values
    if not missing_counts.empty:
        ax = missing_counts.plot(kind='bar', color='salmon', alpha=0.8)
        
        # Add data labels
        for p in ax.patches:
            ax.annotate(f'{int(p.get_height())}', (p.get_x() + p.get_width() / 2., p.get_height()),
                        ha='center', va='center', xytext=(0, 9), textcoords='offset points', fontsize=10)

        plt.title("Count of Missing Values by Column (Before Cleaning)", fontsize=16)
        plt.ylabel("Number of Missing Rows", fontsize=12)
        plt.xlabel("Column Name", fontsize=12)
        plt.xticks(rotation=45, ha='right')
        plt.grid(axis='y', linestyle='--', alpha=0.7)
        plt.tight_layout()
        plt.show()
    else:
        log_report("No missing values found in any column.")

except Exception as e:
    log_report(f"Error checking for missing data: {e}")

# -----------------------------------------------------------------
# 2. Row Completeness
# -----------------------------------------------------------------
required_cols = ['Store ID', 'Product ID', 'Date', 'Units Sold']  # Adjust as needed
try:
    missing_required = df_check[df_check[required_cols].isna().any(axis=1)].index
    problem_rows['missing_required'].update(missing_required)
    log_report(f"Rows missing required columns: {len(missing_required)}")

    # --- Enhanced Plotting ---
    plt.figure(figsize=(10, 5))
    missing_required_counts = df_check[required_cols].isna().sum()
    missing_required_counts = missing_required_counts[missing_required_counts > 0]
    if not missing_required_counts.empty:
        ax = missing_required_counts.plot(kind='bar', color='skyblue', alpha=0.8)

        # Add data labels
        for p in ax.patches:
            ax.annotate(f'{int(p.get_height())}', (p.get_x() + p.get_width() / 2., p.get_height()),
                        ha='center', va='center', xytext=(0, 9), textcoords='offset points', fontsize=10)

        plt.title("Missing Values in Required Columns", fontsize=16)
        plt.ylabel("Number of Missing Rows", fontsize=12)
        plt.xlabel("Column Name", fontsize=12)
        plt.xticks(rotation=45, ha='right')
        plt.grid(axis='y', linestyle='--', alpha=0.7)
        plt.tight_layout()
        plt.show()
    else:
        log_report("No missing values found in required columns.")

except Exception as e:
    log_report(f"Error checking for missing required columns: {e}")

# -----------------------------------------------------------------
# 3. Data Type Verification
# -----------------------------------------------------------------
try:
    log_report("Data Types:\n" + str(df_check.dtypes))
    
    # --- Enhanced Plotting ---
    plt.figure(figsize=(10, 5))
    dtype_counts = df_check.dtypes.value_counts()
    if not dtype_counts.empty:
        ax = dtype_counts.plot(kind='bar', color='limegreen', alpha=0.8)

        # Add data labels
        for p in ax.patches:
            ax.annotate(f'{int(p.get_height())}', (p.get_x() + p.get_width() / 2., p.get_height()),
                        ha='center', va='center', xytext=(0, 9), textcoords='offset points', fontsize=10)

        plt.title("Distribution of Column Data Types After Import", fontsize=16)
        plt.ylabel("Number of Columns", fontsize=12)
        plt.xlabel("Data Type", fontsize=12)
        plt.xticks(rotation=0)
        plt.grid(axis='y', linestyle='--', alpha=0.7)
        plt.tight_layout()
        plt.show()
    else:
        log_report("Could not determine data types.")

except Exception as e:
    log_report(f"Error checking data types: {e}")

# -----------------------------------------------------------------
# 4. Unique Identifiers (Composite Key)
# -----------------------------------------------------------------
try:
    duplicate_composite = df_check[df_check.duplicated(subset=['Store ID', 'Product ID', 'Date'], keep=False)].index
    problem_rows['duplicate_composite'].update(duplicate_composite)
    log_report(f"Duplicate records by composite key: {len(duplicate_composite)}")
except Exception as e:
    log_report(f"Error checking for duplicate rows: {e}")

# -----------------------------------------------------------------
# 5. Category/Label Consistency
# -----------------------------------------------------------------
cat_cols = {
    'Category': 'rare_category',
    'Region': 'rare_region',
    'Weather Condition': 'rare_weather',
    'Seasonality': 'rare_seasonality',
}
threshold = 5  # Adjust as needed
try:
    for col, key in cat_cols.items():
        # Logic for finding rare labels remains the same
        value_counts = df_check[col].value_counts()
        rare_labels = value_counts[value_counts < threshold].index
        rare_rows = df_check[df_check[col].isin(rare_labels)].index
        problem_rows[key].update(rare_rows)
        log_report(f"Rows with rare '{col}' labels (count < {threshold}): {len(rare_rows)}")

        # --- Enhanced Plotting ---
        plt.figure(figsize=(12, 6))
        
        # For clarity, only plot top 20 most frequent labels
        plot_counts = value_counts.head(20)
        
        ax = plot_counts.plot(kind='bar', color='purple', alpha=0.8)

        # Add data labels
        for p in ax.patches:
            ax.annotate(f'{int(p.get_height())}', (p.get_x() + p.get_width() / 2., p.get_height()),
                        ha='center', va='center', xytext=(0, 9), textcoords='offset points', fontsize=9)

        title_suffix = ""
        if len(value_counts) > 20:
            title_suffix = " (Top 20 Shown)"

        plt.title(f"Distribution of '{col}' Labels{title_suffix}", fontsize=16)
        plt.ylabel("Number of Records", fontsize=12)
        plt.xlabel(f"{col} Label", fontsize=12)
        plt.xticks(rotation=45, ha='right')
        plt.grid(axis='y', linestyle='--', alpha=0.7)
        plt.tight_layout()
        plt.show()

except Exception as e:
    log_report(f"Error checking category/label consistency: {e}")

# -----------------------------------------------------------------
# 6. Date Validity
# -----------------------------------------------------------------
try:
    # Invalid dates (NaT after parsing)
    invalid_date = df_check[df_check['Date'].isna()].index
    problem_rows['invalid_date'].update(invalid_date)
    log_report(f"Rows with invalid dates: {len(invalid_date)}")
    
    # Future dates
    today = pd.Timestamp.today().normalize()
    future_date = df_check[df_check['Date'] > today].index
    problem_rows['future_date'].update(future_date)
    log_report(f"Rows with future dates: {len(future_date)}")
    
    # Ancient dates (adjust threshold as needed)
    ancient_date = df_check[df_check['Date'] < pd.Timestamp('2000-01-01')].index
    problem_rows['ancient_date'].update(ancient_date)
    log_report(f"Rows with ancient dates: {len(ancient_date)}")

    # --- Enhanced Plotting: Date Coverage Heatmap ---
    if 'Date' in df_check.columns and not df_check['Date'].isna().all():
        # Create a pivot table for the heatmap
        df_check['Year'] = df_check['Date'].dt.year
        df_check['Month'] = df_check['Date'].dt.month
        date_heatmap = df_check.pivot_table(index='Year', columns='Month', values='Store ID', aggfunc='count').fillna(0)

        plt.figure(figsize=(14, 7))
        sns.heatmap(date_heatmap, cmap='viridis', annot=True, fmt='.0f', linewidths=.5)
        
        plt.title("Monthly Transaction Heatmap", fontsize=16)
        plt.xlabel("Month", fontsize=12)
        plt.ylabel("Year", fontsize=12)
        plt.tight_layout()
        plt.show()

        # Clean up temporary columns
        df_check = df_check.drop(columns=['Year', 'Month'])
    else:
        log_report("Skipping date heatmap due to missing or invalid date column.")

except Exception as e:
    log_report(f"Error checking date validity: {e}")

# -----------------------------------------------------------------
# 7. Value Validation
# -----------------------------------------------------------------
try:
    negative_inventory = df_check[df_check['Inventory Level'] < 0].index
    problem_rows['negative_inventory'].update(negative_inventory)
    log_report(f"Rows with negative inventory: {len(negative_inventory)}")
    
    negative_price = df_check[df_check['Price'] < 0].index
    problem_rows['negative_price'].update(negative_price)
    log_report(f"Rows with negative price: {len(negative_price)}")
    
    sales_exceeds_inventory = df_check[df_check['Units Sold'] > df_check['Inventory Level']].index
    problem_rows['sales_exceeds_inventory'].update(sales_exceeds_inventory)
    log_report(f"Rows where sales exceed inventory: {len(sales_exceeds_inventory)}")

    # --- Enhanced Plotting ---
    value_issues = {
        'Negative Inventory': len(negative_inventory),
        'Negative Price': len(negative_price),
        'Sales > Inventory': len(sales_exceeds_inventory)
    }
    value_issues_series = pd.Series(value_issues)
    value_issues_series = value_issues_series[value_issues_series > 0]

    if not value_issues_series.empty:
        plt.figure(figsize=(10, 5))
        ax = value_issues_series.plot(kind='bar', color='orange', alpha=0.8)

        # Add data labels
        for p in ax.patches:
            ax.annotate(f'{int(p.get_height())}', (p.get_x() + p.get_width() / 2., p.get_height()),
                        ha='center', va='center', xytext=(0, 9), textcoords='offset points', fontsize=10)

        plt.title("Count of Rows Failing Initial Value Validation", fontsize=16)
        plt.ylabel("Number of Rows", fontsize=12)
        plt.xlabel("Validation Check", fontsize=12)
        plt.xticks(rotation=0)
        plt.grid(axis='y', linestyle='--', alpha=0.7)
        plt.tight_layout()
        plt.show()
    else:
        log_report("No value validation issues found.")

except Exception as e:
    log_report(f"Error checking value validation: {e}")

# -----------------------------------------------------------------
# 8. Flag Consistency
# -----------------------------------------------------------------
try:
    non_binary_promotion = df_check[~df_check['Promotion'].isin([True, False, 0, 1])].index
    problem_rows['non_binary_promotion'].update(non_binary_promotion)
    log_report(f"Rows with non-binary Promotion: {len(non_binary_promotion)}")
    
    non_binary_epidemic = df_check[~df_check['Epidemic'].isin([True, False, 0, 1])].index
    problem_rows['non_binary_epidemic'].update(non_binary_epidemic)
    log_report(f"Rows with non-binary Epidemic: {len(non_binary_epidemic)}")

    # --- Enhanced Plotting ---
    for flag in ['Promotion', 'Epidemic']:
        plt.figure(figsize=(8, 4))
        flag_counts = df_check[flag].value_counts()
        
        ax = flag_counts.plot(kind='bar', color='cornflowerblue', alpha=0.8)

        # Add data labels
        for p in ax.patches:
            ax.annotate(f'{int(p.get_height())}', (p.get_x() + p.get_width() / 2., p.get_height()),
                        ha='center', va='center', xytext=(0, 9), textcoords='offset points', fontsize=10)

        plt.title(f"Distribution of '{flag}' Flag", fontsize=16)
        plt.ylabel("Number of Records", fontsize=12)
        plt.xlabel("Flag Value", fontsize=12)
        plt.xticks(rotation=0)
        plt.grid(axis='y', linestyle='--', alpha=0.7)
        plt.tight_layout()
        plt.show()

except Exception as e:
    log_report(f"Error checking flag consistency: {e}")

# -----------------------------------------------------------------
# 9. Final Consolidated Report
# -----------------------------------------------------------------
print("\n" + "="*80)
print("A. Data Quality Checks: Consolidated Report")
print("="*80)
print(report_text)
print("All problematic row indices have been stored in the 'problem_rows' dictionary.")

# ================================================================
# Restore index for time series operations (optional)
# ================================================================
df = df_check.set_index('Date').sort_index()

In [ ]:
# ==========================================================================
# Step B.1: Granularity, Coverage, and Data Integrity Checks
# Purpose: To ensure the dataset is complete, consistent, and fit for forecasting by analyzing its
#          granularity, coverage, and identifying critical integrity issues like duplicates and value anomalies.
# Outputs: A printed audit log, a dictionary of problematic rows, and summary visualizations.
# Rationale: This transparent, progressive check supports actionable data cleaning by systematically
#            validating the core dimensions and integrity of the data before advanced analysis.
# ==========================================================================

import pandas as pd
import matplotlib.pyplot as plt

# Assume df is your DataFrame, with 'Date' as index and columns:
# 'Product ID', 'Category', 'Store ID', 'Region', 'Inventory Level', 'Units Sold', 'Price', 'Promotion', 'Epidemic'

# Initialize log as a list (to be printed at the end)
log = []
log.append("=== DATA VALIDATION AUDIT LOG ===\n\n")

# Store problematic rows as pandas Series or DataFrames
problematic_rows = {}

# ==========================================================================
# Define validation rules and metrics
# ==========================================================================
validation_rules = {
    'Inventory Level': {'min': 0, 'max': 10000},
    'Price': {'min': 0, 'max': 1000},
    'Units Sold': {'min': 0, 'max': 1000},
    'Promotion': {'allowed': [True, False, 1, 0]},
    'Epidemic': {'allowed': [True, False, 1, 0]}
}
required_cols = ['Product ID', 'Category', 'Store ID', 'Region', 'Inventory Level', 'Units Sold', 'Price', 'Promotion', 'Epidemic']

try:
    # ==========================================================================
    # Automated data quality checks
    # ==========================================================================
    missing_cols = [col for col in required_cols if col not in df.columns]
    if missing_cols:
        log.append(f"Missing required columns: {missing_cols}\n")
        problematic_rows['missing_columns'] = missing_cols
    else:
        log.append("All required columns present.\n")

    missing_values = df[required_cols].isna().sum()
    if missing_values.sum() > 0:
        log.append("Missing values detected:\n")
        log.append(missing_values.to_string() + "\n")
        problematic_rows['missing_values'] = missing_values
    else:
        log.append("No missing values found in required columns.\n")

    # ==========================================================================
    # Change index to column for duplicate check on composite key
    # This creates a temporary DataFrame with 'Date' as a column, not the index
    # ==========================================================================
    df_with_date = df.reset_index()
    composite_key = ['Store ID', 'Product ID', 'Date']
    duplicates = df_with_date[df_with_date.duplicated(subset=composite_key, keep=False)]
    if not duplicates.empty:
        log.append(f"Duplicate rows by composite key (Store/Product/Date): {len(duplicates)}\n")
        problematic_rows['duplicates'] = duplicates
    else:
        log.append("No duplicate rows found by composite key (Store/Product/Date).\n")

    # ==========================================================================
    # Check for possible duplicate rows by date only (if date is index)
    # This helps identify days with suspiciously high record counts
    # ==========================================================================
    date_duplicates = df.groupby(df.index).size()
    suspicious_dates = date_duplicates[date_duplicates > date_duplicates.median() + 2 * date_duplicates.std()]
    if not suspicious_dates.empty:
        log.append(f"\nPossible duplicate or suspiciously high record counts by date:\n")
        log.append(f"Found {len(suspicious_dates)} dates with unusually high record counts.\n")
        log.append("Top dates with high record counts:\n")
        log.append(suspicious_dates.sort_values(ascending=False).head().to_string() + "\n")
        problematic_rows['suspicious_dates'] = suspicious_dates
    else:
        log.append("No suspiciously high record counts by date found.\n")

    for field, rules in validation_rules.items():
        if field in df.columns:
            if 'min' in rules:
                below_min = df[df[field] < rules['min']]
                if not below_min.empty:
                    log.append(f"Rows with {field} below minimum ({rules['min']}): {len(below_min)}\n")
                    problematic_rows[f'{field}_below_min'] = below_min[field]
            if 'max' in rules:
                above_max = df[df[field] > rules['max']]
                if not above_max.empty:
                    log.append(f"Rows with {field} above maximum ({rules['max']}): {len(above_max)}\n")
                    problematic_rows[f'{field}_above_max'] = above_max[field]
            if 'allowed' in rules:
                invalid = df[~df[field].isin(rules['allowed'])]
                if not invalid.empty:
                    log.append(f"Rows with invalid {field}: {len(invalid)}\n")
                    problematic_rows[f'invalid_{field}'] = invalid[field]

    # ==========================================================================
    # Granularity and coverage analysis
    # ==========================================================================
    metrics = {
        "Product Count": df['Product ID'].nunique(),
        "Category Count": df['Category'].nunique(),
        "Date Range": (df.index.min(), df.index.max()),
        "Store Count": df['Store ID'].nunique(),
        "Region Count": df['Region'].nunique()
    }

    log.append("\n=== DATASET DIMENSIONS ===\n")
    for key, val in metrics.items():
        if isinstance(val, tuple):
            log.append(f"{key}: {val[0].date()} to {val[1].date()}\n")
        else:
            log.append(f"{key}: {val}\n")

    # ==========================================================================
    # Actionable insights and remediation (context-aware, only if issues found)
    # ==========================================================================
    log.append("\n=== ACTIONABLE INSIGHTS ===\n")
    any_issues = False

    if 'missing_columns' in problematic_rows:
        log.append("1. Review and correct missing required columns: see 'missing_columns'.\n")
        any_issues = True
    if 'missing_values' in problematic_rows:
        log.append("2. Review and correct missing values: see 'missing_values'.\n")
        any_issues = True
    if 'duplicates' in problematic_rows:
        log.append("3. Investigate and resolve duplicate records: see 'duplicates'.\n")
        any_issues = True
    if 'suspicious_dates' in problematic_rows:
        log.append("4. Investigate suspiciously high record counts by date: see 'suspicious_dates'.\n")
        any_issues = True

    # Field-specific checks
    for field in ['Inventory Level', 'Price', 'Units Sold', 'Promotion', 'Epidemic']:
        if f'{field}_below_min' in problematic_rows:
            log.append(f"5. Check and correct rows with {field} below minimum: see '{field}_below_min'.\n")
            any_issues = True
        if f'{field}_above_max' in problematic_rows:
            log.append(f"6. Check and correct rows with {field} above maximum: see '{field}_above_max'.\n")
            any_issues = True
        if f'invalid_{field}' in problematic_rows:
            log.append(f"7. Validate and standardize {field} values: see 'invalid_{field}'.\n")
            any_issues = True

    if not any_issues:
        log.append("No data quality issues found. No corrective actions required.\n")

    # ==========================================================================
    # Statistical and visual analysis (print plots to notebook)
    # ==========================================================================
    try:
        # --- Enhanced Daily Record Count Plot ---
        plt.figure(figsize=(12, 6))
        daily_counts = df.resample('D').size()
        
        # Plot daily counts
        daily_counts.plot(label='Daily Record Count', color='skyblue', alpha=0.8)
        
        # Plot 30-day rolling average
        daily_counts.rolling(window=30).mean().plot(label='30-Day Rolling Average', color='darkblue', linestyle='--')

        plt.title('Daily Record Count and 30-Day Rolling Average', fontsize=16)
        plt.ylabel('Number of Records', fontsize=12)
        plt.xlabel('Date', fontsize=12)
        plt.legend()
        plt.grid(axis='y', linestyle='--', alpha=0.7)
        plt.tight_layout()
        plt.show()

        # --- Enhanced Units Sold Distribution Plot ---
        plt.figure(figsize=(12, 6))
        sns.histplot(df['Units Sold'], kde=True, bins=50, color='lightcoral', alpha=0.8)
        
        # Add mean and median lines
        mean_units_sold = df['Units Sold'].mean()
        median_units_sold = df['Units Sold'].median()
        plt.axvline(mean_units_sold, color='red', linestyle='--', label=f'Mean: {mean_units_sold:.2f}')
        plt.axvline(median_units_sold, color='green', linestyle='-', label=f'Median: {median_units_sold:.2f}')

        plt.title('Distribution of Units Sold (All Records)', fontsize=16)
        plt.xlabel('Units Sold', fontsize=12)
        plt.ylabel('Frequency', fontsize=12)
        plt.legend()
        plt.grid(axis='y', linestyle='--', alpha=0.7)
        plt.tight_layout()
        plt.show()
    except Exception as e:
        log.append(f"Error during visualization: {e}\n")

except Exception as e:
    log.append(f"Critical error during validation: {e}\n")

# ==========================================================================
# Print log to cell output
# ==========================================================================
print(''.join(log))
print("\nData validation checks completed. Problematic rows are stored in the 'problematic_rows' dictionary.")

# Optional: Print problematic rows for review
if problematic_rows:
    print("\n=== PROBLEMATIC ROWS ===\n")
    for key, rows in problematic_rows.items():
        print(f"{key}:")
        print(rows if isinstance(rows, (pd.Series, pd.DataFrame)) else rows)
        print()
else:
    print("\nNo problematic rows found.")

In [ ]:
# ==========================================================================
# Step B.2: Advanced Temporal Integrity Checks
# Purpose: To perform deeper validation of the dataset's time-series structure by checking for
#          temporal continuity, identifying out-of-bounds timestamps, and flagging suspicious record volumes.
# Outputs: A printed summary report, a dictionary of problematic rows, and actionable insights for remediation.
# Rationale: High-standard temporal validation is essential for reliable time-series forecasting, as gaps
#            or anomalies can significantly skew model performance.
# ==========================================================================

from datetime import datetime

# Helper function to format date lists
def format_dates(dates):
    if hasattr(dates, 'empty') and dates.empty:
        return 'None'
    elif len(dates) == 0:
        return 'None'
    return ', '.join([d.strftime('%Y-%m-%d') for d in dates])

# 1. Date Sequence Completeness
expected_dates = pd.date_range(start=df.index.min(), end=df.index.max(), freq='D')
missing_dates = expected_dates.difference(df.index)
missing_dates_list = list(missing_dates)

# 2. Logical Timestamp Ranges
today = datetime.now().date()
future_dates = df.index[df.index > pd.Timestamp(today)]
ancient_threshold = pd.Timestamp('2000-01-01')  # Adjust as needed
ancient_dates = df.index[df.index < ancient_threshold]

# 3. Suspiciously High or Low Record Counts by Date
date_counts = df.groupby(df.index).size()
suspicious_dates = date_counts[
    (date_counts > date_counts.median() + 2 * date_counts.std()) |
    (date_counts < date_counts.median() - 2 * date_counts.std())
]

# 4. Dataset Dimension Recap (for reporting)
dimension_metrics = {
    "Product Count": df['Product ID'].nunique(),
    "Category Count": df['Category'].nunique(),
    "Date Range": (df.index.min().date(), df.index.max().date()),
    "Store Count": df['Store ID'].nunique(),
    "Region Count": df['Region'].nunique()
}

# 5. Print Summary Report
print("""
=== STEP B.2: ADVANCED DATASET DIMENSION & TEMPORAL INTEGRITY CHECKS ===

--- Dataset Dimensions ---
Product Count:   {Product Count}
Category Count:  {Category Count}
Date Range:      {Date Range[0]} to {Date Range[1]}
Store Count:     {Store Count}
Region Count:    {Region Count}

--- Temporal Integrity ---
Missing dates in sequence: {missing_dates}
  Details: {missing_dates_details}
Future dates detected: {future_dates}
  Details: {future_dates_details}
Ancient dates detected: {ancient_dates}
  Details: {ancient_dates_details}
Dates with suspicious record counts: {suspicious_dates}
  Details: {suspicious_dates_details}
""".format(
    **dimension_metrics,
    missing_dates=len(missing_dates_list),
    missing_dates_details=format_dates(missing_dates_list),
    future_dates=len(future_dates),
    future_dates_details=format_dates(future_dates),
    ancient_dates=len(ancient_dates),
    ancient_dates_details=format_dates(ancient_dates),
    suspicious_dates=len(suspicious_dates),
    suspicious_dates_details=suspicious_dates.sort_values(ascending=False).head().to_string() if not suspicious_dates.empty else 'None'
))

# 6. Store problematic results for downstream review
problematic_rows['b2_missing_dates'] = pd.Series(missing_dates_list)
problematic_rows['b2_future_dates'] = pd.Series(future_dates)
problematic_rows['b2_ancient_dates'] = pd.Series(ancient_dates)
problematic_rows['b2_suspicious_dates'] = suspicious_dates

# 7. Actionable Insights
if len(missing_dates_list) > 0 or len(future_dates) > 0 or len(ancient_dates) > 0 or len(suspicious_dates) > 0:
    print("\n=== ACTIONABLE INSIGHTS (B.2) ===\n")
    if len(missing_dates_list) > 0:
        print("• Fill or impute missing dates for time series continuity (see 'b2_missing_dates').")
        print(problematic_rows['b2_missing_dates'].head())
    if len(future_dates) > 0 or len(ancient_dates) > 0:
        print("• Correct or flag future/ancient dates (see 'b2_future_dates', 'b2_ancient_dates').")
        print("Future:", problematic_rows['b2_future_dates'].head())
        print("Ancient:", problematic_rows['b2_ancient_dates'].head())
    if len(suspicious_dates) > 0:
        print("• Investigate dates with suspicious record counts (see 'b2_suspicious_dates').")
        print(problematic_rows['b2_suspicious_dates'].head())
else:
    print("\nNo advanced temporal integrity issues found in dataset dimensions.")

In [ ]:
# ==========================================================================
# Step B.3: Raw Data Import Sanity Check
# Purpose: To import the dataset in its original, unaltered state to validate the effectiveness
#          and necessity of the automated cleaning steps.
# Outputs: A separate "raw" DataFrame and a printed report comparing its quality issues against the cleaned version.
# Rationale: This check provides a definitive baseline of the source data's quality and confirms that
#            the cleaning process is correctly identifying and resolving issues.
# ==========================================================================

import pandas as pd
import numpy as np
from pathlib import Path
import logging

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Data configuration
DATA_PATH = Path('/kaggle/input/retail-store-inventory-and-demand-forecasting/sales_data.csv')
DATE_COL = 'Date'
DTYPES = {
    'Store ID': 'category',
    'Product ID': 'category',
    'Category': 'category',
    'Region': 'category',
    'Inventory Level': 'uint16',
    'Units Sold': 'uint16',
    'Units Ordered': 'uint16',
    'Price': 'float32',
    'Discount': 'float32',
    'Weather Condition': 'category',
    'Promotion': 'bool',
    'Competitor Pricing': 'float32',
    'Seasonality': 'category',
    'Epidemic': 'bool',
    'Demand': 'uint16'
}

def import_raw_data():
    """Import data WITHOUT any corrections to assess original data quality."""
    try:
        logger.info("Importing raw data without corrections...")
        print("Importing RAW data (no corrections applied)...")
        raw_df = pd.read_csv(
            DATA_PATH,
            dtype=DTYPES,
            parse_dates=[DATE_COL],
            on_bad_lines='warn',
            encoding='utf-8'
        )
        raw_df = raw_df.set_index(DATE_COL).sort_index()
        logger.info(f"Raw data imported: {len(raw_df):,} rows")
        print(f"Raw data imported: {len(raw_df):,} rows")
        return raw_df
    except FileNotFoundError:
        logger.error(f"Data file not found: {DATA_PATH}")
        print(f"ERROR: Data file not found: {DATA_PATH}")
    except pd.errors.DtypeWarning as e:
        logger.warning(f"Data type conversion warning: {e}")
        print(f"WARNING: Data type conversion warning: {e}")
    except Exception as e:
        logger.error(f"Unexpected error importing raw data: {e}")
        print(f"ERROR: Unexpected error importing raw data: {e}")
    return None

def check_raw_data_quality(raw_df):
    """Perform data quality checks on the raw data."""
    print("\n" + "="*60)
    print("RAW DATA QUALITY ISSUES (Before Corrections)")
    print("="*60)
    issues_found = []

    # 1. Negative inventory
    try:
        negative_inv = (raw_df['Inventory Level'] < 0).sum()
        if negative_inv > 0:
            issues_found.append(f"Negative Inventory: {negative_inv:,} rows")
            print(f"ISSUE: Negative Inventory: {negative_inv:,} rows")
            logger.warning(f"Negative Inventory: {negative_inv:,} rows")
        else:
            print("CLEAN: Negative Inventory: 0 rows")
    except Exception as e:
        logger.error(f"Error checking negative inventory: {e}")
        print(f"ERROR: Could not check negative inventory: {e}")

    # 2. Sales exceeding inventory
    try:
        sales_exceed = (raw_df['Units Sold'] > raw_df['Inventory Level']).sum()
        if sales_exceed > 0:
            issues_found.append(f"Sales > Inventory: {sales_exceed:,} rows")
            print(f"ISSUE: Sales > Inventory: {sales_exceed:,} rows")
            logger.warning(f"Sales > Inventory: {sales_exceed:,} rows")
        else:
            print("CLEAN: Sales > Inventory: 0 rows")
    except Exception as e:
        logger.error(f"Error checking sales vs inventory: {e}")
        print(f"ERROR: Could not check sales vs inventory: {e}")

    # 3. Negative prices
    try:
        negative_price = (raw_df['Price'] < 0).sum()
        if negative_price > 0:
            issues_found.append(f"Negative Prices: {negative_price:,} rows")
            print(f"ISSUE: Negative Prices: {negative_price:,} rows")
            logger.warning(f"Negative Prices: {negative_price:,} rows")
        else:
            print("CLEAN: Negative Prices: 0 rows")
    except Exception as e:
        logger.error(f"Error checking negative prices: {e}")
        print(f"ERROR: Could not check negative prices: {e}")

    # 4. Missing values
    try:
        missing_total = raw_df.isnull().sum().sum()
        if missing_total > 0:
            issues_found.append(f"Missing Values: {missing_total:,} total")
            print(f"ISSUE: Missing Values: {missing_total:,} total")
            logger.warning(f"Missing Values: {missing_total:,} total")
        else:
            print("CLEAN: Missing Values: 0 total")
    except Exception as e:
        logger.error(f"Error checking missing values: {e}")
        print(f"ERROR: Could not check missing values: {e}")

    # 5. Invalid dates (index nulls)
    try:
        date_issues = raw_df.index.isnull().sum()
        if date_issues > 0:
            issues_found.append(f"Invalid Dates: {date_issues:,} rows")
            print(f"ISSUE: Invalid Dates: {date_issues:,} rows")
            logger.warning(f"Invalid Dates: {date_issues:,} rows")
        else:
            print("CLEAN: Invalid Dates: 0 rows")
    except Exception as e:
        logger.error(f"Error checking date validity: {e}")
        print(f"ERROR: Could not check date validity: {e}")

    # 6. Check for lists/arrays in Category or other columns
    # (You can add more columns to this list if needed)
    cat_cols = ['Category', 'Region', 'Store ID', 'Product ID', 'Weather Condition', 'Seasonality']
    for col in cat_cols:
        try:
            is_list = raw_df[col].apply(lambda x: isinstance(x, (list, tuple, np.ndarray)))
            if is_list.any():
                issues_found.append(f"Lists/arrays in {col}: {is_list.sum():,} rows")
                print(f"ISSUE: Lists/arrays in {col}: {is_list.sum():,} rows")
                logger.warning(f"Lists/arrays in {col}: {is_list.sum():,} rows")
                print("Sample of problematic rows:")
                display(raw_df[is_list].head())
            else:
                print(f"CLEAN: Lists/arrays in {col}: 0 rows")
        except Exception as e:
            logger.error(f"Error checking lists/arrays in {col}: {e}")
            print(f"ERROR: Could not check lists/arrays in {col}: {e}")

    return issues_found

def report_sanity_check_results(issues_found, raw_df, df):
    print("\n" + "="*60)
    print("SANITY CHECK RESULTS")
    print("="*60)
    if issues_found:
        print("VALIDATION: Raw data contains the following quality issues:")
        for issue in issues_found:
            print(f"  - {issue}")
        print("\nThese issues were addressed by the automatic correction process.")
        logger.info(f"Data quality issues detected: {issues_found}")
    else:
        print("RESULT: No data quality issues detected in the raw data based on current checks.")
        print("Interpretation:")
        print("  - The dataset may be synthetic or pre-cleaned for the challenge.")
        print("  - Data type coercion or import settings are unlikely to mask issues, but can be reviewed if needed.")
        print("  - If real-world data is expected, further investigation is recommended.")
        logger.info("No data quality issues found in raw data.")

    print("\nComparison with corrected data:")
    print(f"Raw data rows: {len(raw_df):,}")
    print(f"Corrected data rows: {len(df):,}")
    print(f"Row difference: {len(df) - len(raw_df):,}")

    # Show sample of raw data
    print("\nRaw data sample:")
    display(raw_df.head())

    # Key statistics comparison
    print("\nKey statistics comparison (Raw vs Corrected):")
    comparison_cols = ['Inventory Level', 'Units Sold', 'Price']
    for col in comparison_cols:
        try:
            print(f"\n{col}:")
            print(f"  Raw       - Min: {raw_df[col].min():.2f}, Max: {raw_df[col].max():.2f}")
            print(f"  Corrected - Min: {df[col].min():.2f}, Max: {df[col].max():.2f}")
        except Exception as e:
            logger.error(f"Error comparing statistics for {col}: {e}")
            print(f"ERROR: Could not compare statistics for {col}: {e}")

# Main execution
raw_df = import_raw_data()
if raw_df is not None:
    issues_found = check_raw_data_quality(raw_df)
    # 'df' should be your corrected/processed DataFrame, assumed to be loaded elsewhere
    try:
        report_sanity_check_results(issues_found, raw_df, df)
    except Exception as e:
        logger.error(f"Error during reporting: {e}")
        print(f"ERROR: Could not complete reporting: {e}")
else:
    logger.error("Failed to import raw data for comparison")
    print("FAILED: Could not import raw data for comparison")

## Advanced Data Consistency & Plausibility Checks

**Purpose**

While the raw data passes all standard quality checks (negatives, missing values, logical errors, etc.), subtle issues may still exist, especially in synthetic datasets. To ensure robust modeling and reliable insights, we perform additional advanced checks. The previous approach focused on price and category consistency across regions. Given the observed wild price variation and to deepen our understanding, we now also examine:

- **Pricing consistency within each region by store and item**
- **Category consistency within each region by item**
- **Price and category consistency across all regions**

These checks help uncover hidden artifacts, mislabeling, or implausible economic patterns that could impact model performance and interpretation.

---

**Stages of Advanced Checks**

**Stage 1: Pricing Consistency Within Each Region by Store and Item**  
*Check if the same product shows plausible price variation within each region across different stores. Detect unrealistic uniformity or outliers, and compare pricing patterns with competitors within the region. This exposes extreme values, pricing errors, and ensures that price deviations are justifiable (e.g., due to promotions or local market conditions).*

**Stage 2: Category Consistency Within Each Region**  
*Ensure each product is consistently assigned to the same category in all records within each region. Flag any mislabeled products or inconsistent categorization within a region.*

**Stage 3: Price vs. Competitor Pricing Analysis Within Regions**  
*Analyze the relationship between store prices and competitor pricing within each region. Identify implausible gaps, artificial patterns, or deviations from expected pricing strategies.*

**Stage 4: Item Category Mapping Consistency Across All Regions**  
*Ensure each product is mapped to the same category across all regions. Flag any products with inconsistent category assignments, which could indicate mislabeling or data entry errors.*

---

In [ ]:
# ==========================================================================
# Step C.1: Intra-Regional Price Consistency Check
# Purpose: To ensure that price variation for each product across different stores within the same region is plausible.
# Outputs: An audit log, a dictionary of problematic row indices, and visualizations of price distributions.
# Rationale: This check helps detect data artifacts, pricing errors, or unrealistic uniformity by verifying that
#            price deviations are statistically justifiable (e.g., due to promotions or local market conditions).
# ==========================================================================

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import logging

# Configure logging for traceability
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# ==========================================================================
# Helper: IQR-based outlier detection
# ==========================================================================
def iqr_outliers(data, k=1.5):
    """Detect outliers using the IQR method. Returns a boolean mask of outliers."""
    q1 = np.percentile(data, 25)
    q3 = np.percentile(data, 75)
    iqr = q3 - q1
    lower_bound = q1 - k * iqr
    upper_bound = q3 + k * iqr
    return (data < lower_bound) | (data > upper_bound)

# ==========================================================================
# Main check: Intra-regional price consistency
# ==========================================================================
def check_pricing_within_region(
    df,
    region_col='Region',
    store_col='Store ID',
    product_col='Product ID',
    price_col='Price',
    competitor_col='Competitor Pricing'
):
    """
    Check for plausible price variation within each region by store and item.
    Detects unrealistic uniformity, outliers, and compares with competitor prices.
    Logs errors, stores problematic row indices, and provides statistics and visualizations.
    Returns pd.Index of problematic rows.
    """
    log = []
    log.append("=== INTRA-REGIONAL PRICE CONSISTENCY AUDIT LOG ===\n\n")
    problematic_row_indices = pd.Index([])

    try:
        # ==========================================================================
        # Check for required columns
        # ==========================================================================
        required_cols = [region_col, store_col, product_col, price_col]
        if competitor_col in df.columns:
            required_cols.append(competitor_col)
        missing_cols = [col for col in required_cols if col not in df.columns]
        if missing_cols:
            log.append(f"Missing required columns: {missing_cols}\n")
            logger.error(f"Missing required columns: {missing_cols}")
            raise ValueError(f"Missing required columns: {missing_cols}")
        log.append("All required columns present.\n")

        # ==========================================================================
        # Group data by region, product, and store for price analysis
        # ==========================================================================
        log.append("Grouping data by region, product, and store...\n")
        grouped = df.groupby([region_col, product_col, store_col], observed=True)[price_col].mean().reset_index()

        # ==========================================================================
        # Analyze price variation and log issues
        # ==========================================================================
        uniformity_issues = []
        outlier_issues = []
        for (region, product), group in grouped.groupby([region_col, product_col], observed=True):
            prices = group[price_col]
            n_stores = len(prices)
            if n_stores > 1:
                # Outlier detection
                outliers = iqr_outliers(prices.values)
                if outliers.any():
                    outlier_stores = group[store_col][outliers].tolist()
                    outlier_prices = prices[outliers].tolist()
                    outlier_issues.append((region, product, outlier_stores, outlier_prices))
                    for store in outlier_stores:
                        mask = (df[region_col] == region) & (df[product_col] == product) & (df[store_col] == store)
                        problematic_row_indices = problematic_row_indices.union(df[mask].index)
                # Uniformity detection
                if prices.nunique() == 1:
                    uniformity_issues.append((region, product, prices.iloc[0]))

        # ==========================================================================
        # Statistical summary
        # ==========================================================================
        log.append("\n=== STATISTICAL SUMMARY ===\n")
        stats = grouped.groupby([region_col, product_col], observed=True)[price_col].agg(['count', 'mean', 'std', 'min', 'max'])
        log.append(stats.to_string() + "\n")

        # ==========================================================================
        # Uniformity and outlier issues summary
        # ==========================================================================
        log.append("\n=== UNIFORMITY ISSUES ===\n")
        if uniformity_issues:
            uniform_df = pd.DataFrame(uniformity_issues, columns=['Region', 'Product', 'Price'])
            log.append(f"Products with identical prices in all stores within a region: {len(uniformity_issues)}\n")
            log.append(uniform_df.groupby('Region').size().rename('Uniform Products Count').to_string() + "\n")
            log.append("Sample:\n" + uniform_df.head(10).to_string() + "\n")
        else:
            log.append("No products with identical prices in all stores within a region.\n")

        log.append("\n=== OUTLIER ISSUES ===\n")
        if outlier_issues:
            log.append(f"Products with outlier prices in some stores within a region: {len(outlier_issues)}\n")
            for i, (region, product, stores, prices) in enumerate(outlier_issues[:10]):
                log.append(f"Region {region}, Product {product}, Stores {stores}, Prices {prices}\n")
        else:
            log.append("No products with outlier prices in some stores within a region.\n")

        # ==========================================================================
        # Problematic row indices summary
        # ==========================================================================
        log.append("\n=== PROBLEMATIC ROWS ===\n")
        if len(problematic_row_indices) > 0:
            log.append(f"Total problematic rows: {len(problematic_row_indices)}\n")
            log.append(f"Problematic rows index (Date):\n{problematic_row_indices}\n")
        else:
            log.append("No problematic rows detected.\n")

        # ==========================================================================
        # Visual analysis with seaborn
        # ==========================================================================
        try:
            if len(grouped) > 0:
                # Boxplot: Price distribution by product in a sample region
                sample_region = grouped[region_col].iloc[0]
                sample_df = grouped[grouped[region_col] == sample_region]
                plt.figure(figsize=(12, 6))
                sns.boxplot(data=sample_df, x=product_col, y=price_col)
                plt.title(f'Price Distribution by Product in Region {sample_region}')
                plt.xticks(rotation=45)
                plt.tight_layout()
                plt.show()

                # Scatterplot: Price vs Competitor Price (if available)
                if competitor_col in df.columns:
                    sample_df = df[df[region_col] == sample_region]
                    plt.figure(figsize=(12, 6))
                    sns.scatterplot(data=sample_df, x=price_col, y=competitor_col, hue=product_col)
                    plt.title(f'Price vs Competitor Price in Region {sample_region}')
                    plt.tight_layout()
                    plt.show()
        except Exception as e:
            log.append(f"Error during visualization: {e}\n")

    except Exception as e:
        log.append(f"Critical error during intra-regional price consistency check: {e}\n")
        logger.error(f"Critical error during intra-regional price consistency check: {e}")

    # ==========================================================================
    # Print audit log to cell output
    # ==========================================================================
    print(''.join(log))
    print("\nIntra-regional price consistency checks completed. Problematic row indices are stored in 'problematic_row_indices'.")

    return problematic_row_indices

# ==========================================================================
# Run the check and store problematic row indices for review
# ==========================================================================
problematic_row_indices = check_pricing_within_region(
    df,
    region_col='Region',
    store_col='Store ID',
    product_col='Product ID',
    price_col='Price',
    competitor_col='Competitor Pricing'
)

# ==========================================================================
# Optional: Print problematic rows for review
# ==========================================================================
if len(problematic_row_indices) > 0:
    print("\n=== PROBLEMATIC ROWS ===\n")
    print(df.loc[problematic_row_indices])
else:
    print("\nNo problematic rows found.")

In [ ]:
# ==========================================================================
# Step C.2: Intra-Regional Category Consistency Check
# Purpose: To ensure each product is consistently assigned to the same category in all records within a given region.
# Outputs: An audit log and a dictionary of row indices for any products with inconsistent category assignments.
# Rationale: This check prevents ambiguity and misrepresentation of products at the regional level, ensuring that
#            regional analysis and modeling are based on accurate category data.
# ==========================================================================

def check_category_consistency(df, region_col='Region', product_col='Product ID', category_col='Category'):
    """
    Check for consistent category assignment within each region and product.
    Log inconsistencies and return problematic row indices.
    """
    log = []
    inconsistent_indices = pd.Index([])
    log.append("=== CATEGORY CONSISTENCY WITHIN REGION AUDIT LOG ===\n\n")

    try:
        # Group by region and product, count unique categories
        group = df.groupby([region_col, product_col])[category_col].nunique().reset_index()
        inconsistent = group[group[category_col] > 1]
        
        if len(inconsistent) > 0:
            log.append(f"Found {len(inconsistent)} product-region pairs with inconsistent category assignment.\n")
            log.append("Sample of inconsistent pairs:\n")
            log.append(inconsistent.head(10).to_string() + "\n")
            
            # Store indices of problematic rows
            for _, row in inconsistent.iterrows():
                mask = (df[region_col] == row[region_col]) & (df[product_col] == row[product_col])
                inconsistent_indices = inconsistent_indices.union(df[mask].index)
        else:
            log.append("No inconsistent category assignments found within any region.\n")
    except Exception as e:
        log.append(f"Error during category consistency check: {e}\n")
    
    print(''.join(log))
    return inconsistent_indices

# Run the check
inconsistent_category_indices = check_category_consistency(
    df,
    region_col='Region',
    product_col='Product ID',
    category_col='Category'
)

# Optional: Print problematic rows for review
if len(inconsistent_category_indices) > 0:
    print("\n=== PROBLEMATIC ROWS ===\n")
    print(df.loc[inconsistent_category_indices])
else:
    print("\nNo problematic rows found.")

## Next Steps: Correcting Category Mislabeling

## **Findings Summary**

- **Inconsistent Category Assignments:**  
  - 16 product-region pairs were identified with inconsistent category assignments.
  - Example: Multiple categories (e.g., "Electronics" and "Clothing") assigned to the same product within a region.
- **Problematic Rows:**  
  - Specific rows with inconsistent labels have been flagged and can be reviewed or corrected as needed.

## **Action Plan**

1. **Majority Vote Correction:**  
   - For each product-region pair, determine the most frequently assigned category (the mode).
   - Update all records for that product-region pair to use the majority category.



2. **Manual Review for Edge Cases:**  
   - For cases where the majority is unclear or validation is inconclusive, flag for manual review.

3. **Update the Dataset:**  
   - Replace inconsistent category labels with the corrected value.
   - Ensure all changes are documented for traceability.

4. **Re-run Consistency Checks:**  
   - Rerun the category consistency check to confirm that all issues have been resolved.
   - Repeat the process if any inconsistencies remain.

In [ ]:
# ==========================================================================
# Step C.2.1: Correct Intra-Regional Category Assignments
# Purpose: To automatically correct inconsistent product category assignments within each region by applying a majority vote.
# Outputs: An updated DataFrame with corrected category labels and a confirmation that no inconsistencies remain.
# Rationale: This automated correction ensures that each product has a single, unambiguous category within its region,
#            resolving data conflicts before proceeding to cross-regional analysis.
# ==========================================================================



# ==========================================================================
# 1. Majority Vote Correction
# ==========================================================================
# Group by region and product, find the most common category for each group
majority_category = (
    df.groupby(['Region', 'Product ID'])['Category']
    .apply(lambda x: x.mode()[0] if not x.mode().empty else x.iloc[0])
    .reset_index()
    .rename(columns={'Category': 'Corrected_Category'})
)

# Merge the corrected categories back into the original DataFrame
df = df.merge(
    majority_category,
    on=['Region', 'Product ID'],
    how='left'
)

# Replace the original category column with the corrected one
df['Category'] = df['Corrected_Category']
df = df.drop(columns=['Corrected_Category'])


# ==========================================================================
# 2. Manual Review for Edge Cases (Optional)
# ==========================================================================
# For cases where the majority is unclear (e.g., ties or ambiguous features),
# you can flag these for manual review. Example:
# Find groups with multiple mode values (ties)
tie_groups = (
    df.groupby(['Region', 'Product ID'])['Category']
    .apply(lambda x: len(x.mode()) > 1)
    .reset_index(name='Is_Tie')
)
tie_groups = tie_groups[tie_groups['Is_Tie']]
if not tie_groups.empty:
    print("The following product-region pairs have category ties (manual review recommended):")
    print(tie_groups)
else:
    print("No category ties found. All assignments are unambiguous.")

# ==========================================================================
# 3. Update the Dataset
# ==========================================================================
# The DataFrame is already updated above. This step is complete.

# ==========================================================================
# 4. Re-run Consistency Checks
# ==========================================================================
# Re-run your category consistency check to confirm all issues are resolved.
# (Use the same function as before.)
inconsistent_category_indices = check_category_consistency(
    df,
    region_col='Region',
    product_col='Product ID',
    category_col='Category'
)

# Optionally, print problematic rows if any are found
if len(inconsistent_category_indices) > 0:
    print("\n=== PROBLEMATIC ROWS AFTER CORRECTION ===\n")
    print(df.loc[inconsistent_category_indices])
else:
    print("\nNo problematic rows found after correction. Category assignments are now consistent.")

In [ ]:
# ==========================================================================
# Step C.3: Price vs. Competitor Pricing Analysis
# Purpose: To analyze the relationship between store prices and competitor pricing within each region.
# Outputs: A summary of findings, visualizations, and a list of row indices with implausible price gaps or ratios.
# Rationale: This check helps identify artificial patterns, data entry errors, or deviations from expected
#            pricing strategies that could impact model interpretation and accuracy.
# ==========================================================================

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Check if 'Competitor Pricing' column exists
if 'Competitor Pricing' not in df.columns:
    print("No 'Competitor Pricing' column found. Skipping Step 3.")
else:
    # ==========================================================================
    # 1. Price vs. Competitor Price by Region
    # ==========================================================================
    # Calculate price differences and ratios
    df['Price_Diff'] = df['Price'] - df['Competitor Pricing']
    df['Price_Ratio'] = df['Price'] / df['Competitor Pricing']

    # ==========================================================================
    # 2. Identify Implausible Gaps or Deviations
    # ==========================================================================
    # Define thresholds for implausible gaps (adjust as needed)
    price_diff_threshold = 50  # Example: absolute difference threshold
    price_ratio_threshold = 0.5  # Example: ratio threshold (Price is less than half or more than double)
    price_ratio_upper = 2.0     # Example: upper ratio threshold

    # Flag rows with implausible gaps or ratios
    df['Implausible_Gap'] = df['Price_Diff'].abs() > price_diff_threshold
    df['Implausible_Ratio'] = (df['Price_Ratio'] < price_ratio_threshold) | (df['Price_Ratio'] > price_ratio_upper)

    # Combine flags for any implausible condition
    df['Implausible_Pricing'] = df['Implausible_Gap'] | df['Implausible_Ratio']

    # ==========================================================================
    # 3. Summarize and Log Findings
    # ==========================================================================
    print("=== PRICE VS. COMPETITOR PRICING ANALYSIS ===")
    print(f"Found {df['Implausible_Pricing'].sum()} rows with implausible price gaps or ratios.")
    if df['Implausible_Pricing'].sum() > 0:
        print("\nSample of rows with implausible pricing:")
        print(df[df['Implausible_Pricing']].head(10))
    else:
        print("\nNo implausible price gaps or ratios detected.")

    # ==========================================================================
    # 4. Visualize Price vs. Competitor Pricing by Region
    # ==========================================================================
    plt.figure(figsize=(12, 6))
    sns.scatterplot(data=df, x='Price', y='Competitor Pricing', hue='Region', alpha=0.7)
    plt.title('Price vs. Competitor Pricing by Region')
    plt.axline((0, 0), slope=1, color='red', linestyle='--')
    plt.tight_layout()
    plt.show()

    # ==========================================================================
    # 5. Store Indices of Implausible Pricing for Review
    # ==========================================================================
    # Store indices for follow-up (optional)
    implausible_indices = df[df['Implausible_Pricing']].index
    print("\nIndices of rows with implausible pricing (for review):")
    print(implausible_indices)

In [ ]:
# ==========================================================================
# Step C.4: Global Category Consistency Check
# Purpose: To ensure each product is mapped to the same category across all regions, establishing a global standard.
# Outputs: An audit log and a list of product IDs and row indices that have inconsistent category assignments.
# Rationale: This global check is crucial for maintaining data integrity across the entire dataset, ensuring that a
#            product is treated as the same entity regardless of its region, which is vital for accurate modeling.
# ==========================================================================

import pandas as pd

def check_global_category_consistency(df, product_col='Product ID', category_col='Category'):
    """
    Check if each product is assigned to the same category across all regions.
    Log inconsistencies and return problematic product IDs.
    """
    log = []
    log.append("=== GLOBAL CATEGORY MAPPING CONSISTENCY AUDIT LOG ===\n\n")

    # Group by product and count unique categories
    group = df.groupby(product_col)[category_col].nunique().reset_index()
    inconsistent_products = group[group[category_col] > 1]

    if len(inconsistent_products) > 0:
        log.append(f"Found {len(inconsistent_products)} products with inconsistent category assignments across regions.\n")
        log.append("Sample of inconsistent products:\n")
        log.append(inconsistent_products.head(10).to_string() + "\n")

        # Store indices of problematic rows for review
        problematic_products = inconsistent_products[product_col].tolist()
        problematic_indices = df[df[product_col].isin(problematic_products)].index
        log.append(f"Indices of rows with inconsistent global category assignments:\n{problematic_indices}\n")
    else:
        log.append("No products with inconsistent category assignments across regions.\n")
        problematic_indices = pd.Index([])

    print(''.join(log))
    return problematic_indices

# Run the check
global_inconsistent_indices = check_global_category_consistency(
    df,
    product_col='Product ID',
    category_col='Category'
)

# Optional: Print problematic rows for review
if len(global_inconsistent_indices) > 0:
    print("\n=== PROBLEMATIC ROWS FOR GLOBAL CATEGORY CONSISTENCY ===\n")
    print(df.loc[global_inconsistent_indices])
else:
    print("\nNo problematic rows found for global category consistency.")

# Summary: Global Category Mapping Consistency

## **Findings**

- **Inconsistent Category Assignments:**  
  - **19 products** were found to have inconsistent category assignments across regions.
  - **Example:** Product P0001 is assigned to 3 different categories across regions; P0003 to 4 different categories.
  - **Impact:** This inconsistency can lead to errors in reporting, analysis, and modeling, as the same product is treated differently in different regions.

- **Root Cause Analysis:**  
  - **Intra-region consistency** was achieved in Stage 2 (each product has only one category per region).
  - **Inter-region inconsistency** remains (different regions assign different categories to the same product).
  - **Previous corrections did not cause this issue**—they simply revealed a broader data quality challenge.

## **Corrective Steps**

1. **Analyze Category Assignments by Region:**  
   - For each inconsistent product, review the categories assigned in each region.
   - Identify patterns or root causes (e.g., data entry errors, naming conventions, or genuine business rules).

2. **Standardize Category Mappings:**  
   - **Establish a global standard** for each product’s category.
   - **Update all records** to use the agreed-upon category, regardless of region.

3. **Document Region-Specific Exceptions:**  
   - If certain products must have different categories in specific regions for valid business reasons, document these as exceptions.
   - Ensure these exceptions are clearly flagged and justified.

4. **Re-run Consistency Checks:**  
   - After making corrections, re-run the global category consistency check to confirm all issues are resolved.
   - Repeat the process if any inconsistencies remain.

5. **Maintain Ongoing Audits:**  
   - **Regularly audit** category assignments as new products or regions are added.
   - **Update your category taxonomy** as needed to reflect business changes.

## **Why This Matters**

- **Data Integrity:** Ensures that products are treated consistently across all regions, reducing errors in analysis and reporting.
- **Actionable Insights:** Supports accurate demand forecasting, assortment planning, and marketing strategies.
- **Scalability:** Makes it easier to integrate new products and regions into your data pipeline.

In [ ]:
# ==========================================================================
# Step C.4.1: Correct Global Category Assignments
# Purpose: To enforce a single, global category for each product across all regions by assigning the most common (mode) category.
# Outputs: A corrected DataFrame and a final confirmation that all cross-regional category inconsistencies are resolved.
# Rationale: This final correction step establishes a definitive "source of truth" for product categorization, ensuring
#            the highest level of data consistency before feature engineering and modeling.
# ==========================================================================

import pandas as pd

def check_global_category_consistency(df, product_col='Product ID', category_col='Category'):
    """
    Check if each product is assigned to the same category across all regions.
    Log inconsistencies and return problematic product IDs and indices.
    """
    # Group by product and count unique categories
    group = df.groupby(product_col)[category_col].nunique().reset_index()
    inconsistent_products = group[group[category_col] > 1]

    if len(inconsistent_products) > 0:
        print(f"Found {len(inconsistent_products)} products with inconsistent category assignments across regions.")
        print("Sample of inconsistent products:")
        print(inconsistent_products.head(10))
        # Store indices of problematic rows for review
        problematic_products = inconsistent_products[product_col].tolist()
        problematic_indices = df[df[product_col].isin(problematic_products)].index
        print(f"\nIndices of rows with inconsistent global category assignments:\n{problematic_indices}")
    else:
        print("No products with inconsistent category assignments across regions.")
        problematic_indices = pd.Index([])
    return problematic_indices

def fix_global_category_consistency(df, product_col='Product ID', category_col='Category'):
    """
    Ensure each product is mapped to the same category across all regions.
    Assign the most common (mode) category for each product as the global standard.
    """
    # Find the most common (mode) category for each product across all regions
    # If there are ties, take the first mode
    mode_category = (
        df.groupby(product_col)[category_col]
        .apply(lambda x: x.mode()[0] if not x.mode().empty else x.iloc[0])
        .reset_index()
        .rename(columns={category_col: 'Global_Category'})
    )
    
    # Merge the global category back into the original DataFrame
    df = df.merge(
        mode_category,
        on=product_col,
        how='left'
    )
    
    # Replace the original category column with the global category
    df[category_col] = df['Global_Category']
    df = df.drop(columns=['Global_Category'])
    return df

# Apply the correction
df = fix_global_category_consistency(df, product_col='Product ID', category_col='Category')

# Re-run the global category consistency check to confirm all issues are resolved
global_inconsistent_indices = check_global_category_consistency(
    df,
    product_col='Product ID',
    category_col='Category'
)

if len(global_inconsistent_indices) > 0:
    print("\n=== WARNING: PROBLEMATIC ROWS FOR GLOBAL CATEGORY CONSISTENCY ===\n")
    print(df.loc[global_inconsistent_indices])
else:
    print("\nSUCCESS: No problematic rows found for global category consistency after correction.")

In [ ]:
# ==========================================================================
# Step D: Demand Characteristics Analysis
# Purpose: To analyze demand patterns to inform model selection and feature engineering.
# Outputs: Summary statistics and visualizations of demand distribution, zero-demand frequency, and the impact of promotions/events.
# Rationale: Understanding the fundamental characteristics of the demand signal is essential for choosing an appropriate
#            forecasting model and engineering features that capture relevant business dynamics.
# ==========================================================================

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# --- 1. Value Range (Min/Max) ---
print("="*50)
print("Demand (Units Sold): Summary Statistics")
print("="*50)
print(df['Units Sold'].describe())
print("\n")

# --- 2. Zero-Demand Frequency ---
zero_demand = (df['Units Sold'] == 0).sum()
total_rows = len(df)
print(f"Zero-demand rows: {zero_demand} ({zero_demand/total_rows:.2%})")
print("\n")

# --- 3. Epidemic & Promotion Variance ---
print("="*50)
print("Epidemic and Promotion Impact on Demand")
print("="*50)
print("Epidemic periods:", df['Epidemic'].sum(), "rows")
print("Promotion periods:", df['Promotion'].sum(), "rows")
print("\n")

if df['Epidemic'].sum() > 0:
    print("Average Units Sold during Epidemic:", df[df['Epidemic']]['Units Sold'].mean())
    print("Average Units Sold outside Epidemic:", df[~df['Epidemic']]['Units Sold'].mean())
    print("\n")
if df['Promotion'].sum() > 0:
    print("Average Units Sold during Promotion:", df[df['Promotion']]['Units Sold'].mean())
    print("Average Units Sold outside Promotion:", df[~df['Promotion']]['Units Sold'].mean())
    print("\n")

# --- 4. Regional Patterns ---
print("="*50)
print("Regional Demand Patterns")
print("="*50)
print(df.groupby('Region')['Units Sold'].describe())
print("\n")

# --- 5. Intuitive Visualizations ---

# A. Histogram with KDE
plt.figure(figsize=(12, 6))
sns.histplot(df['Units Sold'], kde=True, bins=50, color='skyblue', alpha=0.8)
# Add mean and median lines
mean_units_sold = df['Units Sold'].mean()
median_units_sold = df['Units Sold'].median()
plt.axvline(mean_units_sold, color='red', linestyle='--', label=f'Mean: {mean_units_sold:.2f}')
plt.axvline(median_units_sold, color='green', linestyle='-', label=f'Median: {median_units_sold:.2f}')
plt.title('Distribution of Units Sold (All Regions)', fontsize=16)
plt.xlabel('Units Sold', fontsize=12)
plt.ylabel('Frequency', fontsize=12)
plt.legend()
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

# B. Faceted Histogram by Region
plt.figure(figsize=(14, 6))
g = sns.FacetGrid(df, col='Region', col_wrap=2, height=4)
g.map(sns.histplot, 'Units Sold', kde=True, bins=30, color='lightcoral', alpha=0.8)
g.set_axis_labels("Units Sold", "Frequency")
g.set_titles("Region: {col_name}")
g.fig.suptitle('Distribution of Units Sold by Region', y=1.05, fontsize=16)
plt.tight_layout(rect=[0, 0.03, 1, 0.98]) # Adjust layout to prevent title overlap
plt.show()

# C. Violin Plot by Region
plt.figure(figsize=(12, 6))
sns.violinplot(data=df, x='Region', y='Units Sold', palette='viridis')
plt.title('Units Sold Distribution by Region (Violin Plot)', fontsize=16)
plt.xlabel('Region', fontsize=12)
plt.ylabel('Units Sold', fontsize=12)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

# D. Scatterplot of Demand vs. Forecast (if available)
if 'Demand' in df.columns:
    plt.figure(figsize=(10, 6))
    sns.scatterplot(data=df, x='Demand', y='Units Sold', alpha=0.4, color='darkblue')
    plt.title('Forecast Demand vs. Actual Units Sold', fontsize=16)
    plt.xlabel('Forecast Demand', fontsize=12)
    plt.ylabel('Actual Units Sold', fontsize=12)
    plt.plot([df['Demand'].min(), df['Demand'].max()], [df['Demand'].min(), df['Demand'].max()], 'r--', label='Perfect Forecast')
    
    # Add correlation coefficient as annotation
    correlation = df['Demand'].corr(df['Units Sold'])
    plt.annotate(f'Correlation (R): {correlation:.2f}', xy=(0.05, 0.95), xycoords='axes fraction', fontsize=12, color='red')

    plt.legend()
    plt.grid(axis='both', linestyle='--', alpha=0.7)
    plt.show()

# --- 6. Additional Insights (Optional) ---
if 'Demand' in df.columns:
    print("="*50)
    print("Forecast Demand vs. Actual Units Sold")
    print("="*50)
    print(df[['Demand', 'Units Sold']].describe())
    print("\nCorrelation between Demand and Units Sold:", df['Demand'].corr(df['Units Sold']))
    print("\n")

print("="*50)
print("Part C: Demand Characteristics Analysis Complete")
print("="*50)

## Part C: Demand Characteristics – Summary

**Findings Align with Expectations:**  
The analysis of demand characteristics reveals patterns that are consistent with both business logic and real-world events—notably, the significant impact of epidemic (COVID-19) periods on demand. As expected, units sold are lower during these times, reflecting the disruption caused by the pandemic. Promotions, on the other hand, are shown to effectively increase sales. Demand is relatively stable across regions, with minor variations and some outliers, but no systemic issues are detected.

**Implications for Part 3 (Insights-to-Action Mapping):**  
- **Epidemic Demand Spikes:**  
  - **Action:** Implement shock indicator features in forecasting models to account for COVID-19 disruptions.
- **Promotion Variance:**  
  - **Action:** Continue to monitor and optimize promotion strategies, and include promotion flags as features in models.
- **Regional Patterns:**  
  - **Action:** Maintain regional segmentation in analysis and consider regional dummy variables in models to capture local effects.
- **Zero-Demand Rows:**  
  - **Action:** Validate a sample of zero-demand records to ensure data accuracy, but expect a small number due to intermittent demand.
- **Forecast Accuracy:**  
  - **Action:** Review forecasting methodology to address systematic over-forecasting, especially during non-epidemic periods.

**Conclusion:**  
The demand characteristics are well-explained by known business and external factors. These insights will guide targeted checks and model enhancements in the next phase of the data quality framework.

In [ ]:
# ==========================================================================
# Step E: Final Summary and Documentation
# Purpose: To aggregate and summarize insights from all previous steps, document actions taken, and prepare for reporting.
# Outputs: A collection of Markdown tables and summaries outlining the entire data quality process.
# Rationale: This step provides a comprehensive, high-level overview of the data preparation journey, ensuring
#            transparency and reproducibility for stakeholders and future analysis.
# ==========================================================================

import pandas as pd
from IPython.display import display, Markdown

# --- 1. Data Quality Checks and Cleaning (Steps 1 & 2) ---
summary = {
    'Step': ['1. Data Quality Checks', '2. Data Cleaning'],
    'Description': [
        'Performed checks for missing values, data types, duplicates, negative values, and logical inconsistencies.',
        'Corrected identified issues: filled missing values, fixed negative inventory, ensured category consistency, and addressed value anomalies.'
    ],
    'Key Findings': [
        'Identified and logged issues such as negative inventory, sales > inventory, negative prices, and missing values.',
        'Applied corrections to ensure data integrity and consistency.'
    ]
}

df_summary = pd.DataFrame(summary)
display(df_summary)

# --- 2. Demand Characteristics (Part C) ---
display(Markdown('''
### Demand Characteristics (Part C) – Key Insights

- **Epidemic (COVID-19) periods:** Significantly lower demand, as expected.
- **Promotions:** Effectively increase demand.
- **Regional demand:** Stable, with minor variations and some outliers.
- **Zero-demand rows:** Rare; intermittent demand is not a major issue.
- **Forecast accuracy:** Forecasts tend to over-predict actual sales, but correlation is strong.
- **No major data quality issues were found.**
'''))

# --- 3. Insights-to-Action Mapping (Part 3) ---
display(Markdown('''
## Insights-to-Action Mapping (Part 3)

| Finding                | Impact                              | Action                                      |
|------------------------|-------------------------------------|---------------------------------------------|
| Epidemic demand spikes | Non-stationary time series          | Add shock indicator features to models      |
| Promotion periods      | Increased demand                    | Include promotion flag as a feature         |
| Regional sales clusters| Geographic bias                     | Add region dummy variables                  |
| Zero-demand rows       | Intermittent demand pattern         | Sample-check for accuracy; use intermittent demand models if needed |
| Forecast over-forecasting | Systematic error, excess inventory risk | Review and refine forecasting methodology   |
'''))

# --- 4. Target Outputs (Part 4) ---
display(Markdown('''
## Target Outputs (Part 4)

- **Data Quality Scorecard:** Summary metrics for each quality dimension.
- **Visual Dashboards:** Key findings visualized for stakeholders.
- **Priority Cleaning Tasks:** Roadmap for addressing identified issues.
- **Model Compatibility Assessment:** Evaluation of data suitability for different forecasting models.
- **Feature Engineering Roadmap:** Plan for creating derived features (e.g., region dummies, event flags).
- **Model Selection Justification:** Documentation of model choices based on data characteristics.
'''))

# --- 5. Next Steps ---
display(Markdown('''
## Next Steps

- **Prepare visual dashboards and scorecards.**
- **Implement feature engineering steps (region dummies, event flags).**
- **Refine forecasting methodology based on identified patterns.**
- **Document model selection and justification.**
'''))

print("="*60)
print("Data Quality Process Summary and Documentation Complete")
print("="*60)

In [ ]:
# ==========================================================================
# Step F: Data Quality Scorecard for Model Selection
# Purpose: To create a final scorecard that summarizes all data quality checks, corrections, and key demand
#          characteristics to inform and justify model selection.
# Outputs: A formatted Markdown scorecard presenting key metrics and qualitative factors.
# Rationale: This scorecard serves as a transparent, data-driven foundation for making modeling decisions,
#            linking the quality of the data directly to the forecasting strategy.
# ==========================================================================

import pandas as pd
from IPython.display import display, Markdown

# --- 1. Data Quality Metrics (Pull from previous steps) ---
# (Replace with your actual metrics or calculations as needed)

metrics = {
    'Total Rows': len(df),
    'Missing Values': df.isnull().sum().sum(),
    'Negative Inventory': (df['Inventory Level'] < 0).sum(),
    'Sales > Inventory': (df['Units Sold'] > df['Inventory Level']).sum(),
    'Negative Prices': (df['Price'] < 0).sum(),
    'Zero-Demand Rows': (df['Units Sold'] == 0).sum(),
    'Zero-Demand Percentage': (df['Units Sold'] == 0).mean() * 100
}

# --- 2. Demand Characteristics (Pull from previous steps) ---
demand_stats = {
    'Mean Units Sold': df['Units Sold'].mean(),
    'Std. Units Sold': df['Units Sold'].std(),
    'Epidemic Impact (Avg Units Sold)': df[df['Epidemic']]['Units Sold'].mean(),
    'Non-Epidemic Impact (Avg Units Sold)': df[~df['Epidemic']]['Units Sold'].mean(),
    'Promotion Impact (Avg Units Sold)': df[df['Promotion']]['Units Sold'].mean(),
    'Non-Promotion Impact (Avg Units Sold)': df[~df['Promotion']]['Units Sold'].mean(),
    'Forecast Correlation (vs. Actual)': df['Demand'].corr(df['Units Sold']) if 'Demand' in df.columns else 'N/A'
}

# --- 3. Label Corrections (Tracked during cleaning) ---
label_corrections = {
    'Region': {
        'North ': 'North',
        'South ': 'South'
    },
    'Category': {
        'Electronics': 'Electronics/Appliances',
        'Clothing': 'Apparel'
    }
}

# --- 4. Model Selection Factors ---
model_factors = [
    'Epidemic (COVID-19) periods: Accounted for as external shocks',
    'Promotion periods: Included as features',
    'Regional effects: Regional dummy variables added',
    'Zero-demand patterns: Intermittent demand models considered if needed',
    'Forecast accuracy: Correlation with actual sales is strong, but over-forecasting noted',
    'Label consistency: Region and category labels standardized'
]

# --- 5. Scorecard Presentation ---

display(Markdown('''
# Data Quality Scorecard for Model Selection

---

## 1. Data Quality Metrics

| Metric                  | Value                |
|-------------------------|----------------------|
| Total Rows              | {:,}                |
| Missing Values          | {:,}                |
| Negative Inventory      | {:,}                |
| Sales > Inventory       | {:,}                |
| Negative Prices         | {:,}                |
| Zero-Demand Rows        | {:,}                |
| Zero-Demand Percentage  | {:.2f}%             |

---

## 2. Demand Characteristics

| Aspect                                | Value                |
|----------------------------------------|----------------------|
| Mean Units Sold                       | {:.2f}              |
| Std. Units Sold                       | {:.2f}              |
| Epidemic Impact (Avg Units Sold)      | {:.2f}              |
| Non-Epidemic Impact (Avg Units Sold)  | {:.2f}              |
| Promotion Impact (Avg Units Sold)     | {:.2f}              |
| Non-Promotion Impact (Avg Units Sold) | {:.2f}              |
| Forecast Correlation (vs. Actual)     | {:.2f}              |

---

## 3. Label Corrections

**Summary of changes made to region and category labels:**
'''.format(
    metrics['Total Rows'],
    metrics['Missing Values'],
    metrics['Negative Inventory'],
    metrics['Sales > Inventory'],
    metrics['Negative Prices'],
    metrics['Zero-Demand Rows'],
    metrics['Zero-Demand Percentage'],
    demand_stats['Mean Units Sold'],
    demand_stats['Std. Units Sold'],
    demand_stats['Epidemic Impact (Avg Units Sold)'],
    demand_stats['Non-Epidemic Impact (Avg Units Sold)'],
    demand_stats['Promotion Impact (Avg Units Sold)'],
    demand_stats['Non-Promotion Impact (Avg Units Sold)'],
    demand_stats['Forecast Correlation (vs. Actual)']
)))

for field, changes in label_corrections.items():
    display(Markdown(f'- **{field}**:'))
    for old, new in changes.items():
        display(Markdown(f'  - `{old}` → `{new}`'))

display(Markdown('''
---

## 4. Model Selection Factors

**Key factors considered for model selection:**
'''))
for factor in model_factors:
    display(Markdown(f'- {factor}'))

display(Markdown('''
---

## 5. Next Steps

- **Prepare visual dashboards and scorecards.**
- **Implement feature engineering steps (region dummies, event flags).**
- **Refine forecasting methodology based on identified patterns.**
- **Document model selection and justification.**
'''))

print("="*60)
print("Data Quality Scorecard for Model Selection Complete")
print("="*60)

# **Model Selection and Justification**

---

## 1. Objective

Select a demand forecasting model that:

- Captures **daily, product-level demand dynamics** across stores
- Incorporates **temporal features**, **event indicators**, and **cross-sectional interactions**
- Models "black swan" disruptions (e.g., **COVID-19**) via an `Epidemic` event flag
- Enables **backtesting-based inventory simulation** to assess policy improvements

---

## 2. Temporal and Structural Context

**Rationale:**  
- **Panel time series**: Multiple grouped series (`Store ID` × `Product ID`)  
- **Daily granularity**: Supports modeling short-term volatility and seasonal cycles  
- **Epidemic period**: Binary `Epidemic` flag marks COVID-19–affected days  
- **Exogenous context**: Features such as `Weather`, `Promotion`, `Competitor Pricing`

---

## 3. Industry Case Study Insights

| Source        | Insight                                                             |
|---------------|----------------------------------------------------------------------|
| **Target**    | Modeled pandemic disruptions via event-aware AI, reduced overstock by ~$2B |
| **Walmart**   | Used lag-based, event-enriched forecasting to balance availability vs overstock |
| **JD.com**    | Dynamically retrained models on COVID-affected segments, improved shock resilience |
| **Quantzig**  | Achieved 30%+ accuracy improvement, 20% lower carrying cost using demand simulations |
| **Academia**  | Pandemic-aware LSTM and boosting models consistently outperformed static baselines |

---

## 4. Candidate Models

| Model                          | Strengths                                                | Limitations                                |
|--------------------------------|----------------------------------------------------------|--------------------------------------------|
| ARIMA/SARIMAX                  | Seasonality, trend modeling                              | Weak for multivariate and black swan events|
| Prophet                        | Easy event modeling                                      | Not suitable for high-dimensional panels   |
| RNN / LSTM                     | Captures nonlinear time dependencies                     | Complex, opaque, requires more training data |
| TFT (Temporal Fusion Transformer)| Event-aware, interpretable deep model                  | Computationally intensive, complex tuning  |
| **LightGBM + Temporal Features** | Fast, interpretable, supports categorical + event data  | Requires manual lag/rolling feature design |

---

## 5. Chosen Model: **LightGBM with Temporal and Event Features**

| Justification                             | Notes                                                               |
|-------------------------------------------|---------------------------------------------------------------------|
| Panel-ready                               | Models demand across `Store ID` × `Product ID` simultaneously       |
| Explicit event modeling                   | Includes `Epidemic`, `Promotion`, `Weather`, and seasonality flags |
| Handles structured data efficiently       | Optimized for 76,000 rows, mixed-type variables                     |
| Lag and rolling features supported        | Manual construction enables demand memory and volatility smoothing |
| Interpretable via SHAP values             | Aids post-modeling diagnostics and business communication           |
| Suitable for simulation and backtesting   | Supports scenario-based inventory optimization                     |

---

## 6. Forecasting Strategy

- **Lag Features:** Prior `Units Sold`, `Inventory Level`, `Units Ordered` (t−1, t−7, t−14, t−30)
- **Rolling Averages:** Trailing 7/14/30-day demand windows
- **Temporal Indicators:** `Day of Week`, `Month`, `Seasonality`, `Holiday`
- **Event Flags:** `Epidemic`, `Promotion`, `Weather Condition`
- **Group-Based Encoding:** Mean historical demand by `Store ID` and `Product ID`

---

## 7. Hypothesis

> "A LightGBM-based pipeline augmented with temporal lags, rolling windows, and explicit event indicators will outperform static models by learning cross-store product behavior, capturing COVID-era volatility, and supporting actionable backtesting for inventory adjustment."

---

## 8. Next Steps

1. **Feature engineering**: Lagged, rolling, and time-derived variables  
2. **Model training**: LightGBM with time-aware cross-validation  
3. **Validation**: Error diagnostics by event type (pandemic vs. normal)  
4. **Simulation**: Backtest inventory scenarios using forecasted `Units Sold`  
5. **Policy optimization**: Identify reorder point changes to reduce stockouts and overstock

---

In [ ]:
## Cell 1: Sorting and Setup

1. Sort the dataset by Store ID, Product ID, and Date to ensure consistent temporal ordering before creating time-based features.



In [ ]:
## Cell 2: Lag Features

2. Generate lag features for Units Sold using shifts of 1, 7, and 14 days grouped by Store ID and Product ID.

3. Generate lag features for Inventory Level and Units Ordered using a 1-day shift grouped by Store ID and Product ID.

7. Generate a lagged version of the Stockout flag with a 1-day shift grouped by Store ID and Product ID.



In [ ]:
## Cell 3: Rolling Window Features

4. Compute 7-day rolling mean and standard deviation of Units Sold grouped by Store ID and Product ID, excluding the current day to avoid leakage.

5. Compute 14-day and 30-day rolling statistics for Units Sold and Inventory Level, including max and min demand windows as needed.

12. Calculate a 14-day rolling sum of the Promotion flag grouped by Store ID and Product ID to capture promotional intensity.

13. Calculate a 7-day rolling sum of the Epidemic flag grouped by Store ID and Product ID to quantify local pandemic influence.



In [ ]:
## Cell 4: Price and Stockout Features

6. Create a binary Stockout flag set to 1 if Inventory Level is zero, else 0.

8. Calculate Effective Price as Price multiplied by (1 - Discount / 100).

9. Compute the difference between Effective Price and Competitor Pricing to capture pricing sensitivity.



In [ ]:
## Cell 5: Temporal Features

10. Extract temporal features from the Date index: Day of Week, Month, and a binary IsWeekend flag based on Day of Week.

11. Merge in a holiday calendar and generate a binary IsHoliday flag to mark special retail days.



In [ ]:
## Cell 6: Group-Based Target Encodings

14. Compute the global average Units Sold per Product ID across all stores and merge this as AvgProductSales_global.

15. Compute the global average Units Sold per Store ID across all products and merge this as AvgStoreSales_global.

16. Group by Category and Region to compute the mean Units Sold per Category per Region and merge this as AvgCategorySales_by_Region.

17. Perform mean encoding for Product ID and Store ID based on historical average Units Sold grouped by date or period as appropriate.



In [ ]:
## Cell 7: Finalization and Validation

18. Review all new features to confirm correct alignment with each Store ID, Product ID, and Date.

19. Downcast all new numeric columns to float32 or uint16 where appropriate to optimize memory usage.

20. Save the engineered dataset to file or cache it for training, ensuring that no future data is leaked into any rows used for model input.



In [ ]:
# Model training

In [ ]:
#Validation

In [ ]:
# Simulation

In [ ]:
# Policy optimization